In [4]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split


from sklearn.svm import SVC 


In [ ]:
# iris = datasets.load_iris()
# X = iris["data"][:, (2, 3)] # petal length, petal width
# y = (iris["target"] == 2).astype(np.float64) # Iris-Virginica
# svm_clf = Pipeline((
# ("scaler", StandardScaler()),
# ("linear_svc", LinearSVC(C=1, loss="hinge")),
# ))
# svm_clf.fit(X_scaled, y)

In [ ]:
#FeatureExtract
for i in range(1,13):
    accfile = source + str(i) + acc + '.csv'
    gyrofile = source + str(i) + gyro + '.csv'
    df1 = pd.read_csv(accfile)
    df2 = pd.read_csv(gyrofile)
    ax = df1['ax']
    ay = df1['ay']
    az = df1['az']
    wx = df2['wx']
    wy = df2['wy']
    wz = df2['wz']
    
    Afx, Atx, Ax = signal.spectrogram(ax, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Afy, Aty, Ay = signal.spectrogram(ay, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Afz, Atz, Az = signal.spectrogram(az, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    
    Wfx, Wtx, Wx = signal.spectrogram(wx, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Wfy, Wty, Wy = signal.spectrogram(wy, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Wfz, Wtz, Wz = signal.spectrogram(wz, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    

    Ax = Ax.T
    Ay = Ay.T
    Az = Az.T
    Wx = Wx.T
    Wy = Wy.T
    Wz = Wz.T
    
    resA = np.concatenate((Ax, Ay, Az), axis = 1)
    resW = np.concatenate((Wx, Wy, Wz), axis = 1)
    for j in range(resA.shape[0]):
        sA = sorted(resA[j])
        sW = sorted(resW[j])
        featureAcc = sA[0:50] + sA[resA.shape[1]-50:resA.shape[1]]    #For 100 features set 50 to 25
        featureGyr = sW[0:50] + sW[resA.shape[1]-50:resA.shape[1]]    #For 100 features set 50 to 25
        final = np.concatenate((featureAcc,featureGyr))
        standard_scaler = preprocessing.StandardScaler()
        feature = standard_scaler.fit_transform(final)



In [5]:
#Create sklearn transformer
class FeatureExtractor( BaseEstimator, TransformerMixin):
    
    #ClassConstructor
    def _init_( self  ):
        return
    
    
    #Always returns self
    def fit(self, X, y=None):
        return self;
    
    
    #Method that describes working of transformer
    def transform(self, X, y=None):
        
        ax = X['ax']
        ay = X['ay']
        az = X['az']
        wx = X['wx']
        wy = X['wy']
        wz = X['wz']

        Afx, Atx, Ax = signal.spectrogram(ax, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
        Afy, Aty, Ay = signal.spectrogram(ay, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
        Afz, Atz, Az = signal.spectrogram(az, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)

        Wfx, Wtx, Wx = signal.spectrogram(wx, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
        Wfy, Wty, Wy = signal.spectrogram(wy, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
        Wfz, Wtz, Wz = signal.spectrogram(wz, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)


        Ax = Ax.T
        Ay = Ay.T
        Az = Az.T
        Wx = Wx.T
        Wy = Wy.T
        Wz = Wz.T
    
        resA = np.concatenate((Ax, Ay, Az), axis = 1)
        resW = np.concatenate((Wx, Wy, Wz), axis = 1)
        for j in range(resA.shape[0]):
            sA = sorted(resA[j])
            sW = sorted(resW[j])
            featureAcc = sA[0:50] + sA[resA.shape[1]-50:resA.shape[1]]    #For 100 features set 50 to 25
            featureGyr = sW[0:50] + sW[resA.shape[1]-50:resA.shape[1]]    #For 100 features set 50 to 25
            X = np.concatenate((featureAcc,featureGyr))
            


        return X.values
    
    

In [7]:
preprocess_pipeline = Pipeline(steps = [('spectro_feature_extractor',FeatureExtractor())
                                        ,('std_scaler',StandardScaler())])

full_pipeline = Pipeline(steps = [('preprocess_pipeline',preprocess_pipeline),
                                 ('svm_clf',SVC(kernel = 'linear', C=10))])